# Dependencies

In [ ]:
# ! pip install torch==1.8.1 torchvision==0.9.1 wandb pandas
!pip install wandb --upgrade
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# Set run variable

In [ ]:
# Model config ======
RUN_NAME        = 'unetv1'
N_CLASSES       = 1
INPUT_SIZE      = 128
EPOCHS          = 10
LEARNING_RATE   = 0.0001
START_FRAME     = 16
DROP_RATE       = 0.5

# Data config =======
SAVE_PATH       = './'
DATA_PATH       = './'
IMAGE_PATH      = 'train/images/'
MASK_PATH       = 'train/masks/'

RANDOM_SEED     = 42
VALID_RATIO     = 0.2
BATCH_SIZE      = 16
NUM_WORKERS     = 0
CLASSES         = {1:'salt'}

# Download dataset & setup env variable

In [ ]:
!unzip ../input/tgs-salt-identification-challenge/train.zip -d ./train
clear_output()

In [ ]:
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, dataloader, random_split

from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import wandb
import os
import time

# login wandb
os.environ['WANDB_API_KEY'] = '2a291fe931b1a2be33e1c09cb5b86dcd6843ea48'

In [ ]:
from distutils.dir_util import copy_tree

fromDir = '../input/tgs-salt-identification-challenge'
toDir = './'

copy_tree(fromDir, toDir)

# Create dataset & Dataloader

In [ ]:
class TGSDataset(Dataset):
    """TGS Salt Identification dataset."""
    
    def __init__(self, root_dir=DATA_PATH, transform=None):
        """
        Args:
            root_path (string): Directory with all the images.
            transformer (function): whether to apply the data augmentation scheme
                mentioned in the paper. Only applied on the train split.
        """

        # load dataset from root dir
        train_df  = pd.read_csv(root_dir+'train.csv', index_col='id')
        depths_df = pd.read_csv(root_dir+'depths.csv', index_col='id')
        train_df = train_df.join(depths_df)

        self.root_dir   = root_dir
        self.ids        = train_df.index
        self.depths     = train_df['z'].to_numpy()
        self.rle        = train_df['rle_mask'].to_numpy()
        
        if transform is None:
            self.transfrom = transforms.Compose([transforms.Resize((INPUT_SIZE, INPUT_SIZE)), 
                                                  transforms.Grayscale(), 
                                                  transforms.ToTensor(),])

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        id    = self.ids[index]
        depth = self.depths[index]

        # file should be unzipped
        image = Image.open(self.root_dir+IMAGE_PATH+id+'.png')
        mask  = Image.open(self.root_dir+MASK_PATH+id+'.png')
    
        image = self.transfrom(image)
        mask  = self.transfrom(mask)

        return image, mask

In [ ]:
def get_dataloader(dataset, 
                    batch_size=BATCH_SIZE, random_seed=RANDOM_SEED, 
                    valid_ratio=VALID_RATIO, shuffle=True, num_workers=NUM_WORKERS):
    """
    Params:
    -------
    - dataset: the dataset.
    - batch_size: how many samples per batch to load.
    - random_seed: fix seed for reproducibility.
    - valid_ratio: percentage split of the training set used for
      the validation set. Should be a float in the range [0, 1].
    - shuffle: whether to shuffle the train/validation indices.
    - num_workers: number of subprocesses to use when loading the dataset.
    """

    error_msg = "[!] valid_ratio should be in the range [0, 1]."
    assert ((valid_ratio >= 0) and (valid_ratio <= 1)), error_msg

    # split the dataset
    n = len(dataset)
    n_valid = int(valid_ratio*n)
    n_train = n - n_valid

    # init random seed
    torch.manual_seed(random_seed)

    train_dataset, valid_dataset = random_split(dataset, (n_train, n_valid))

    train_loader = DataLoader(train_dataset, batch_size, shuffle=shuffle, num_workers=num_workers)
    valid_loader = DataLoader(valid_dataset, batch_size, shuffle=False, num_workers=num_workers)

    return train_loader, valid_loader


In [ ]:
# load dataset
dataset = TGSDataset(DATA_PATH)
trainloader, validloader = get_dataloader(dataset=dataset)

In [ ]:
def show_dataset(dataset, n_sample=4):
    """Visualize dataset with n_sample"""
    fig = plt.figure()

    # show image
    for i in range(n_sample):
        image, mask = dataset[i]
        image = transforms.ToPILImage()(image)
        mask = transforms.ToPILImage()(mask)
        print(i, image.size, mask.size)


        plt.tight_layout()
        ax = plt.subplot(1, n_sample, i + 1)
        ax.set_title('Sample #{}'.format(i))
        ax.axis('off')

        plt.imshow(image, cmap="Greys")
        plt.imshow(mask, alpha=0.3, cmap="OrRd")

        if i == n_sample-1:
            plt.show()
            break

In [ ]:
show_dataset(dataset)

# Unet & block

In [ ]:
class BatchActivate(nn.Module):
    def __init__(self, num_features):
        super(BatchActivate, self).__init__()
        self.norm = nn.BatchNorm2d(num_features)

    def forward(self, x):
        return F.relu(self.norm(x))

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel=3, padding=1, stride=1, activation=True):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, 
                            kernel_size=kernel, stride=stride, padding=padding)
        self.batchnorm  = BatchActivate(out_channels)
        self.activation = activation

    def forward(self, x):
        x = self.conv(x)
        if self.activation:
            x = self.batchnorm(x)
        return x

class DoubleConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel=3, padding=1, stride=1):
        super(DoubleConvBlock, self).__init__()
        self.conv1 = ConvBlock(in_channels, out_channels, kernel, padding, stride)
        self.conv2 = ConvBlock(out_channels, out_channels, kernel, padding, stride)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        return x

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, batch_activation=False):
        super(ResidualBlock, self).__init__()
        self.batch_activation = batch_activation
        self.norm  = nn.BatchNorm2d(num_features=in_channels)
        self.conv1 = ConvBlock(in_channels, in_channels, kernel=3, stride=1, padding=1)
        self.conv2 = ConvBlock(in_channels, in_channels, kernel=3, stride=1, padding=1, activation=False)

    def forward(self, x):
        residual = x
        x = self.norm(x)
        x = self.conv1(x)
        x = self.conv2(x)

        x += residual
        # x = x.view(x.size(0),-1)
        
        if self.batch_activation:
            x = self.norm(x)
        
        return x


## Unet original

In [ ]:
class UNet(nn.Module):
    def __init__(self, in_channels=1, n_classes=N_CLASSES, start_fm=START_FRAME):
        super(UNet, self).__init__()
        # Input 1x128x128

        # Maxpool 
        self.pool = nn.MaxPool2d((2,2))

        # Transpose conv
        self.deconv_4  = nn.ConvTranspose2d(start_fm*16, start_fm*8, 2, 2)
        self.deconv_3  = nn.ConvTranspose2d(start_fm*8, start_fm*4, 2, 2)
        self.deconv_2  = nn.ConvTranspose2d(start_fm*4, start_fm*2, 2, 2)
        self.deconv_1  = nn.ConvTranspose2d(start_fm*2, start_fm, 2, 2)
        
        # Encoder 
        self.encoder_1 = DoubleConvBlock(in_channels, start_fm, kernel=3)
        self.encoder_2 = DoubleConvBlock(start_fm, start_fm*2, kernel=3)
        self.encoder_3 = DoubleConvBlock(start_fm*2, start_fm*4, kernel=3)
        self.encoder_4 = DoubleConvBlock(start_fm*4, start_fm*8, kernel=3)

        # Middle
        self.middle = DoubleConvBlock(start_fm*8, start_fm*16)
        
        # Decoder
        self.decoder_4 = DoubleConvBlock(start_fm*16, start_fm*8)
        self.decoder_3 = DoubleConvBlock(start_fm*8, start_fm*4)
        self.decoder_2 = DoubleConvBlock(start_fm*4, start_fm*2)
        self.decoder_1 = DoubleConvBlock(start_fm*2, start_fm)

        self.conv_last = nn.Conv2d(start_fm, n_classes, 1)

    def forward(self, x):
        # Encoder
        conv1 = self.encoder_1(x)
        x     = self.pool(conv1)

        conv2 = self.encoder_2(x)
        x     = self.pool(conv2)

        conv3 = self.encoder_3(x)
        x     = self.pool(conv3)

        conv4 = self.encoder_4(x)
        x     = self.pool(conv4)

        # Middle
        x     = self.middle(x)

        # Decoder
        x     = self.deconv_4(x)
        x     = torch.cat([conv4, x], dim=1)
        x     = self.decoder_4(x)

        x     = self.deconv_3(x)
        x     = torch.cat([conv3, x], dim=1)
        x     = self.decoder_3(x)

        x     = self.deconv_2(x)
        x     = torch.cat([conv2, x], dim=1)
        x     = self.decoder_2(x)

        x     = self.deconv_1(x)
        x     = torch.cat([conv1, x], dim=1)
        x     = self.decoder_1(x)
        
        out   = self.conv_last(x)
        return out

## Unet Res

In [ ]:
class UNet_ResNet(nn.Module):
    def __init__(self, in_channels=1, n_classes=N_CLASSES, dropout=0.5, start_fm=START_FRAME):
        super(UNet_ResNet, self).__init__()

        # Encoder 
        self.encoder_1 = nn.Sequential(
            nn.Conv2d(in_channels, start_fm, 3, padding=(1,1)),
            ResidualBlock(start_fm),
            ResidualBlock(start_fm, batch_activation=True),
            nn.MaxPool2d((2,2)),
            nn.Dropout2d(dropout//2),
        )

        self.encoder_2 = nn.Sequential(
            nn.Conv2d(start_fm, start_fm*2, 3, padding=(1,1)),
            ResidualBlock(start_fm*2),
            ResidualBlock(start_fm*2, batch_activation=True),
            nn.MaxPool2d((2,2)),
            nn.Dropout2d(dropout),
        )

        self.encoder_3 = nn.Sequential(
            nn.Conv2d(start_fm*2, start_fm*4, 3, padding=(1,1)),
            ResidualBlock(start_fm*4),
            ResidualBlock(start_fm*4, batch_activation=True),
            nn.MaxPool2d((2,2)),
            nn.Dropout2d(dropout),
        )
        
        self.encoder_4 = nn.Sequential(
            nn.Conv2d(start_fm*4, start_fm*8, 3, padding=(1,1)),
            ResidualBlock(start_fm*8),
            ResidualBlock(start_fm*8, batch_activation=True),
            nn.MaxPool2d((2,2)),
            nn.Dropout2d(dropout),
        )

        self.middle = nn.Sequential(
            nn.Conv2d(start_fm*8, start_fm*16, 3, padding=3//2),
            ResidualBlock(start_fm*16),
            ResidualBlock(start_fm*16, batch_activation=True),
            nn.MaxPool2d((2,2))
        )
        
        # Transpose conv
        self.deconv_4  = nn.ConvTranspose2d(start_fm*16, start_fm*8, 2, 2)
        self.deconv_3  = nn.ConvTranspose2d(start_fm*8, start_fm*4, 2, 2)
        self.deconv_2  = nn.ConvTranspose2d(start_fm*4, start_fm*2, 2, 2)
        self.deconv_1  = nn.ConvTranspose2d(start_fm*2, start_fm, 2, 2)

        # Decoder 
        self.decoder_4 = nn.Sequential(
            nn.Dropout2d(dropout),
            nn.Conv2d(start_fm*16, start_fm*8, 3, padding=(1,1)),
            ResidualBlock(start_fm*8),
            ResidualBlock(start_fm*8, batch_activation=True),
        )

        self.decoder_3 = nn.Sequential(
            nn.Dropout2d(dropout),
            nn.Conv2d(start_fm*8, start_fm*4, 3, padding=(1,1)),
            ResidualBlock(start_fm*4),
            ResidualBlock(start_fm*4, batch_activation=True),
        )

        self.decoder_2 = nn.Sequential(
            nn.Dropout2d(dropout),
            nn.Conv2d(start_fm*4, start_fm*2, 3, padding=(1,1)),
            ResidualBlock(start_fm*2),
            ResidualBlock(start_fm*2, batch_activation=True),
        )

        self.decoder_1 = nn.Sequential(
            nn.Dropout2d(dropout),
            nn.Conv2d(start_fm*2, start_fm, 3, padding=(1,1)),
            ResidualBlock(start_fm),
            ResidualBlock(start_fm, batch_activation=True),
            nn.ConvTranspose2d(start_fm, start_fm, 2, 2)
        )
            
        self.conv_last = nn.Conv2d(start_fm, n_classes, 1)

    def forward(self, x):
        # Encoder
        conv1 = self.encoder_1(x)

        conv2 = self.encoder_2(conv1)

        conv3 = self.encoder_3(conv2)

        conv4 = self.encoder_4(conv3)

        # Middle
        x     = self.middle(conv4)

        # Decoder
        x     = self.deconv_4(x)
        x     = torch.cat([conv4, x], dim=1)
        x     = self.decoder_4(x)

        x     = self.deconv_3(x)
        x     = torch.cat([conv3, x], dim=1)
        x     = self.decoder_3(x)

        x     = self.deconv_2(x)
        x     = torch.cat([conv2, x], dim=1)
        x     = self.decoder_2(x)

        x     = self.deconv_1(x)
        x     = torch.cat([conv1, x], dim=1)
        x     = self.decoder_1(x)
        
        out   = (self.conv_last(x))
        return out

# Start train
## Helper function

In [ ]:
def labels():
  l = {}
  for i, label in enumerate(CLASSES):
    l[i] = label
  return l

def tensor2np(tensor):
    tensor = tensor.squeeze().cpu()
    return tensor.detach().numpy()

def normtensor(tensor):
    tensor = torch.where(tensor<0., torch.zeros(1).cuda(), torch.ones(1).cuda())
    return tensor

def wandb_mask(bg_imgs, pred_masks, true_masks):
    # bg_imgs    = [np.array(transforms.ToPILImage()(image)) for image in bg_imgs]
    # pred_masks = [np.array(transforms.ToPILImage()(image)) for image in pred_masks]
    # true_masks = [np.array(transforms.ToPILImage()(image)) for image in true_masks]

    return wandb.Image(bg_imgs, masks={
        "predictions" : {
            "mask_data" : pred_masks,
            "class_labels" : CLASSES
            },
        "ground_truth" : {
            "mask_data" : true_masks, 
            "class_labels" : CLASSES
            }
        })
    
def count_params(model):
    pytorch_total_params = sum(p.numel() for p in model.parameters())
    return pytorch_total_params

## Train & Eval function

In [ ]:
def cal_iou(outputs, labels, SMOOTH=1e-6):
    with torch.no_grad():
        outputs = outputs.squeeze(1).bool()  # BATCH x 1 x H x W => BATCH x H x W
        labels = labels.squeeze(1).bool()
        
        intersection = (outputs & labels).float().sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
        union = (outputs | labels).float().sum((1, 2))         # Will be zzero if both are 0
        
        iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
        
        # thresholded = torch.clamp(20 * (iou - 0.5), 0, 10).ceil() / 10  # This is equal to comparing with thresolds
    
    return iou

    # return iou.cpu().detach().numpy()

def get_iou_score(outputs, labels):
    A = labels.squeeze(1).bool()
    pred = torch.where(outputs<0., torch.zeros(1).cuda(), torch.ones(1).cuda())
    B = pred.squeeze(1).bool()
    intersection = (A & B).float().sum((1,2))
    union = (A| B).float().sum((1, 2)) 
    iou = (intersection + 1e-6) / (union + 1e-6)  
    
    return iou.cpu().detach().numpy()

In [ ]:
def train(model, device, trainloader, optimizer, loss_function):
    model.train()
    running_loss = 0
    mask_list, iou = [], []
    for i, (input, mask) in enumerate(trainloader):
        # load data into cuda
        input, mask = input.to(device), mask.to(device)

        # forward
        predict = model(input)
        loss = loss_function(predict, mask)

        # metric
        iou.append(get_iou_score(predict, mask).mean())
        running_loss += (loss.item())
        
        # zero the gradient + backprpagation + step
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        # log the first image of the batch
        if ((i + 1) % 10) == 0:
            pred = normtensor(predict[0])
            img, pred, mak = tensor2np(input[0]), tensor2np(pred), tensor2np(mask[0])
            mask_list.append(wandb_mask(img, pred, mak))
            
    mean_iou = np.mean(iou)
    total_loss = running_loss/len(trainloader)
    wandb.log({'Train loss': total_loss, 'Train IoU': mean_iou, 'Train prediction': mask_list})

    return total_loss, mean_iou

In [ ]:
def test(model, device, testloader, loss_function, best_iou):
    model.eval()
    running_loss = 0
    mask_list, iou  = [], []
    with torch.no_grad():
        for i, (input, mask) in enumerate(testloader):
            input, mask = input.to(device), mask.to(device)

            predict = model(input)
            loss = loss_function(predict, mask)

            running_loss += loss.item()
            iou.append(get_iou_score(predict, mask).mean())

            # log the first image of the batch
            if ((i + 1) % 1) == 0:
                pred = normtensor(predict[0])
                img, pred, mak = tensor2np(input[0]), tensor2np(pred), tensor2np(mask[0])
                mask_list.append(wandb_mask(img, pred, mak))

    test_loss = running_loss/len(testloader)
    mean_iou = np.mean(iou)
    wandb.log({'Valid loss': test_loss, 'Valid IoU': mean_iou, 'Prediction': mask_list})
    
    if mean_iou>best_iou:
    # export to onnx + pt
        try:
            torch.onnx.export(model, input, SAVE_PATH+RUN_NAME+'.onnx')
            torch.save(model.state_dict(), SAVE_PATH+RUN_NAME+'.pth')
        except:
            print('Can export weights')

    return test_loss, mean_iou

In [ ]:
def model_pipeline(config):
    # tell wandb to get started
    with wandb.init(project="TGS-Salt-identification", tags=['Unet'], config=config):
        # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config
        
        # make the model, data, and optimization problem
        model, criterion, optimizer = make(config)
        
        best_iou = -1
        wandb.watch(model, criterion, log="all", log_freq=10)
        for epoch in range(config.epoch):

            t0 = time.time()
            train_loss, train_iou = train(model, device, trainloader, optimizer, criterion)
            t1 = time.time()

            print(f'Epoch: {epoch} | Train loss: {train_loss:.3f} | Train IoU: {train_iou:.3f} | Time: {(t1-t0):.1f}s')

            test_loss, test_iou = test(model, device, validloader, criterion, best_iou)
            print(f'Epoch: {epoch} | Valid loss: {test_loss:.3f} | Valid IoU: {test_iou:.3f} | Time: {(t1-t0):.1f}s')
            
            # Wandb summary
            if best_iou < test_iou:
                best_iou = test_iou
                wandb.run.summary["best_accuracy"] = best_iou
        
        trained_weight = wandb.Artifact(RUN_NAME, type='weights')
        trained_weight.add_file(SAVE_PATH+RUN_NAME+'.onnx')
        trained_weight.add_file(SAVE_PATH+RUN_NAME+'.pth')
        wandb.log_artifact(trained_weight)

        print("Model saved to Wandb")

    return model

In [ ]:
def make(config):
    # Make the model
    model = UNet().to(device)

    print('Number of parameter:', count_params(model))

    # Make the loss and optimizer
    criterion = nn.BCEWithLogitsLoss()
    optimizer   = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    return model, criterion, optimizer

## Training

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
config = dict(
    lr          = LEARNING_RATE,
    batchsize   = BATCH_SIZE,
    epoch       = EPOCHS,
    model_sf    = START_FRAME,
    device      = device,
)

In [ ]:
model = model_pipeline(config)

# Predict

## Helper function

In [ ]:
!unzip ../input/tgs-salt-identification-challenge/test.zip -d ./test
clear_output()

In [ ]:
TEST_IMAGE_PATH = 'test/images/'

In [ ]:
import pandas as pd
train_df  = pd.read_csv('./'+'train.csv', index_col='id')
depths_df = pd.read_csv('./'+'depths.csv', index_col='id')


In [ ]:
test_df = depths_df.loc[depths_df.index.isin(train_df.index) == False]

In [ ]:
test_df

In [ ]:
class Test_TGSDataset(Dataset):
    """TGS Salt Identification dataset."""
    
    def __init__(self, root_dir=DATA_PATH, transform=None):
        """
        Args:
            root_path (string): Directory with all the images.
            transformer (function): whether to apply the data augmentation scheme
                mentioned in the paper. Only applied on the train split.
        """

        # load dataset from root dir
        train_df  = pd.read_csv(root_dir+'train.csv', index_col='id')
        depths_df = pd.read_csv(root_dir+'depths.csv', index_col='id')
        test_df = depths_df.loc[depths_df.index.isin(train_df.index) == False]

        self.root_dir   = root_dir
        self.ids        = test_df.index
        self.depths     = test_df['z'].to_numpy()
        
        if transform is None:
            self.transfrom = transforms.Compose([transforms.Resize((INPUT_SIZE, INPUT_SIZE)), 
                                                  transforms.Grayscale(), 
                                                  transforms.ToTensor(),])

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        id    = self.ids[index]
        depth = self.depths[index]

        # file should be unzipped
        image = Image.open(self.root_dir+TEST_IMAGE_PATH+id+'.png')
        image = self.transfrom(image)

        return image

In [ ]:
def show_test_dataset(dataset, n_sample=4):
    """Visualize dataset with n_sample"""
    fig = plt.figure()

    # show image
    for i in range(n_sample):
        image = dataset[i]
        image = transforms.ToPILImage()(image)
        print(i, image.size)

        plt.tight_layout()
        ax = plt.subplot(1, n_sample, i + 1)
        ax.set_title('Sample #{}'.format(i))
        ax.axis('off')

        plt.imshow(image, cmap="Greys")

        if i == n_sample-1:
            plt.show()
            break

In [ ]:
test_dataset = Test_TGSDataset(DATA_PATH)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
show_test_dataset(test_dataset)

In [ ]:
predicted_mask = []

In [ ]:
def predict(model, test_dataset, device):
    model.eval()
    predicted_masks = []
    back_transform = transforms.Compose([transforms.Resize((101,101))])
    with torch.no_grad():
        for i, input in enumerate(test_loader):
            input = input.to(device)
            predict = model(input)
            predict = back_transform(predict)
            predict = (predict > 0).type(torch.float)
            predicted_masks.append(predict)
    predicted_masks = torch.cat(predicted_masks)
    return predicted_masks
            
    

In [ ]:
predicted_mask = predict(model, test_dataset, device=device)

In [ ]:
def show_sample_test_result(test_dataset, predicted_mask, n_samples=20):
    """Visualize test sample and corresponding result."""
    plt.rcParams["figure.figsize"] = (20,10)
    back_transform = transforms.Compose([transforms.Resize((101,101))])
    for i in range(n_samples):
        sample = predicted_mask[i]  
        sample = torch.squeeze(sample, dim=0)
        sample = transforms.ToPILImage()(sample)
        X = test_dataset[i]
        X = back_transform(X)
        X = transforms.ToPILImage()(X)
        
        ax = plt.subplot(2, int(n_samples/2), i + 1)
        ax.set_title('Sample #{}'.format(i))
        ax.axis('off')
        plt.imshow(X, cmap="Greys")
        plt.imshow(sample, alpha=0.3, cmap="OrRd")
        if i == n_samples-1:
            plt.show()
            break
        
    
    

In [ ]:
show_sample_test_result(test_dataset, predicted_mask)

In [ ]:
predicted_mask_np = predicted_mask.cpu().data.numpy()

In [ ]:
predicted_mask_np[0]

In [ ]:
def rle_encode(im):
    '''
    im: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = im.flatten(order = 'F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
rle_encode(np.squeeze(predicted_mask_np[0]))

In [ ]:
rle_results = []
for im in predicted_mask_np:
    im = np.squeeze(im)
    im_rle_result = rle_encode(im)
    rle_results.append(im_rle_result)

In [ ]:
submit = pd.DataFrame([list(test_dataset.ids), rle_results]).T
submit.columns = ['id', 'rle_mask']
submit.to_csv('torch_submission.csv', index = False)